In [1]:
import pandas as pd
import numpy as np

Reading in Data

In [2]:
movies = pd.read_csv("../data/ml-1m/movies.dat", sep="::", header = None)
ratings = pd.read_csv("../data/ml-1m/ratings.dat", sep="::", header = None)
users = pd.read_csv("../data/ml-1m/users.dat", sep="::", header = None)

c:\python\python36\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
c:\python\python36\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
c:\python\python36\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel packag

In [3]:
print('shape of movies-matrix:', movies.shape)
print('shape of ratings-matrix:', ratings.shape)
print('shape of users-matrix:', users.shape)

shape of movies-matrix: (3883, 3)
shape of ratings-matrix: (1000209, 4)
shape of users-matrix: (6040, 5)


In [4]:
movies.columns = ["MovieID","MovieTitle","Genre"]
movies.head(5)


,MovieID,MovieTitle,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.columns = ["UserID","MovieID","Rating","TimeStamp"]
ratings.head(5)

,UserID,MovieID,Rating,TimeStamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
users.columns = ["UserID","Sex","AgeGroup","OccupationGroup","ZipCode"]
users.head(5)

,UserID,Sex,AgeGroup,OccupationGroup,ZipCode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [87]:


def glb_avg(data,ind_train):
    global_average_rating = data.Rating[ind_train].mean(0)
    rating_vec = np.repeat(global_average_rating,data.shape[0])
    return(rating_vec)
    
ratings.Rating.mean()

3.581564453029317

In [106]:
global_avg_user = ratings.groupby(["UserID"]) [["Rating"]].mean()#()

def glb_avg_user(data,ind_train):
    global_avg_user = data[ind_train].groupby(["UserID"])[["Rating"]].mean() #.transform(mean)
    
    #FallBack 
    rating_vec = glb_avg(data, ind_train)
    
    for id in global_avg_user.index:
        ind = np.where(data["UserID"]==id)
        rating_vec[ind] = global_avg_user.loc[id]
     
    return(rating_vec)

#~1 in global_avg_user.index   
global_avg_user.head(5)

,Rating
UserID,
1,4.188679
2,3.713178
3,3.901961
4,4.190476
5,3.146465


In [107]:
global_avg_movie = ratings.groupby(["MovieID"])[["Rating"]].mean()

def glb_avg_movie(data,ind_train):
    global_avg_mobie = data[ind_train].groupby(["MovieID"])[["Rating"]].mean() #.transform(mean)
    
    #FallBack 
    rating_vec = glb_avg(data, ind_train)
    
    for id in global_avg_movie.index:
        ind = np.where(data["MovieID"]==id)
        rating_vec[ind] = global_avg_movie.loc[id]
     
    return(rating_vec)

global_avg_movie.head(5)

,Rating
MovieID,
1,4.146846
2,3.201141
3,3.016736
4,2.729412
5,3.006757


In [132]:
np.random.seed(610)
Nfolds = 5

#Create folds grouping vector
Nrep = ratings.shape[0] // Nfolds
a = np.repeat(np.arange(Nfolds),Nrep)
b = np.arange(ratings.shape[0] % Nfolds)
folds_vec = np.concatenate([a,b])
np.random.shuffle(folds_vec)


#initialize error vecs
cols = ["global_train", "global_test", "user_train", "user_test", "movie_train", "movie_test", "combined_train", "combined_test"]
err = pd.DataFrame(index = range(0,Nfolds), columns = cols)

for fold in range(Nfolds):
    ind_test = folds_vec == fold
    ind_train = ~ ind_test
    Ytest = ratings.Rating[ind_test]
    Ytrain = ratings.Rating[ind_train]
      
    gobal_avg_vec = glb_avg(ratings,ind_train)
    movie_avg_vec = glb_avg_movie(ratings,ind_train)
    user_avg_vec = glb_avg_user(ratings,ind_train)
    
    #training errors
    err.global_train[fold] = np.sqrt(np.mean( (Ytrain - gobal_avg_vec[ind_train])**2))
    err.movie_train[fold] = np.sqrt(np.mean( (Ytrain - movie_avg_vec[ind_train])**2))
    err.user_train[fold] = np.sqrt(np.mean( (Ytrain - user_avg_vec[ind_train])**2))
    
    #testing errors
    err.global_test[fold] = np.sqrt(np.mean( (Ytest-gobal_avg_vec[ind_test])**2))
    err.movie_test[fold] = np.sqrt(np.mean( (Ytest-movie_avg_vec[ind_test])**2))
    err.user_test[fold] = np.sqrt(np.mean( (Ytest-user_avg_vec[ind_test])**2))
    
    #linear combination method:
    #training
    #Design matrix
    X1 = user_avg_vec[ind_train]
    X2 = movie_avg_vec[ind_train]
    X3 = np.repeat(1,len(X1))
    X = np.vstack([X1,X2,X3]).T
    
    betas = np.linalg.lstsq(X,Ytrain)[0]
    Ypred = np.dot(X, betas)
    #improving Ypred
    Ypred[Ypred>5] = 5
    Ypred[Ypred<1] = 1
    
    #training error
    err.combined_train[fold] = np.sqrt(np.mean( (Ytrain-Ypred)**2))
    
    #testing
    #Design matrix
    X1 = user_avg_vec[ind_test]
    X2 = movie_avg_vec[ind_test]
    X3 = np.repeat(1,len(X1))
    X = np.vstack([X1,X2,X3]).T
    Ypred = np.dot(X, betas)
    #improving Ypred
    Ypred[Ypred>5] = 5
    Ypred[Ypred<1] = 1

    #test error
    err.combined_test[fold] = np.sqrt(np.mean( (Ytest-Ypred)**2))
    
    print("\nRMSE Fold " + str(fold) + ":")
    print(err.loc[fold])
    #print("Fold " + str(fold) + ": RMSE_train=" + str(err_train[fold]) + "; RMSE_test=" + str(err_test[fold]))
    
    
    




c:\python\python36\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.



Fold 0:
global_train       1.11695
global_test         1.1177
user_train         1.02737
user_test          1.03674
movie_train         0.9749
movie_test        0.973956
combined_train    0.915016
combined_test      0.92012
Name: 0, dtype: object

Fold 1:
global_train       1.11736
global_test        1.11609
user_train         1.02769
user_test          1.03554
movie_train       0.974823
movie_test        0.974265
combined_train    0.914845
combined_test      0.92099
Name: 1, dtype: object

Fold 2:
global_train        1.1169
global_test        1.11791
user_train         1.02788
user_test          1.03448
movie_train       0.974876
movie_test        0.974053
combined_train    0.915423
combined_test     0.918883
Name: 2, dtype: object

Fold 3:
global_train       1.11745
global_test        1.11569
user_train         1.02765
user_test          1.03558
movie_train       0.974778
movie_test        0.974446
combined_train    0.914789
combined_test     0.921206
Name: 3, dtype: object

Fold 4:

In [133]:
err


,global_train,global_test,user_train,user_test,movie_train,movie_test,combined_train,combined_test
0,1.11695,1.1177,1.02737,1.03674,0.9749,0.973956,0.915016,0.92012
1,1.11736,1.11609,1.02769,1.03554,0.974823,0.974265,0.914845,0.92099
2,1.1169,1.11791,1.02788,1.03448,0.974876,0.974053,0.915423,0.918883
3,1.11745,1.11569,1.02765,1.03558,0.974778,0.974446,0.914789,0.921206
4,1.11685,1.11812,1.0278,1.03491,0.97418,0.976835,0.914688,0.92141
